<a href="https://colab.research.google.com/github/zoltan23/Audio-Classification/blob/master/audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python_speech_features
!pip install scipy
!pip install pandas
!pip install keras
!pip install sklearn
!pip install tqdm
!pip install librosa
!apt-get install libsndfile1 -y
!apt-get install git -y

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=2eb16b2c1c605eb5d042225389f5872189c624171142476b96cdfa5c65822816
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.chdir('/content/gdrive/My Drive/music-wav-files')

In [10]:
!mkdir clean samples models

mkdir: cannot create directory ‘clean’: File exists
mkdir: cannot create directory ‘samples’: File exists
mkdir: cannot create directory ‘models’: File exists


In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from scipy.io import wavfile as wav
import numpy as np

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

In [6]:
def extract_features(file_name):

    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        mask = envelope(audio, sample_rate, 0.0005)

        mfccs = librosa.feature.mfcc(y=audio[mask], sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)

        #print("mfccsscaled", mfccsscaled.shape)
    
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    
    return mfccsscaled

def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

NameError: ignored

In [19]:
file_name = os.path.join(os.path.abspath(fulldatasetpath), str(row["fname"]))
class_label = row["label"]
data = extract_features('wavfiles/5eadbc6d.wav')


In [14]:
!ls 

clean  instruments.csv	model  models  pickles	samples  wavfiles


# The following code was extracted from a module that was created to train a CNN model.  It will be called to retrain the model to improve accuracy as more data becomes available from the app.  

In [13]:
fulldatasetpath = 'wavfiles/'

metadata = pd.read_csv('instruments.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():

    file_name = os.path.join(os.path.abspath(fulldatasetpath), str(row["fname"]))

    class_label = row["label"]
    data = extract_features(file_name)
    features.append([data, class_label])

    # Convert into a Panda dataframe 
    featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

    #print('Finished feature extraction from ', len(featuresdf), ' files') 


# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

num_labels = yy.shape[1]
print("num_labels", num_labels)
filter_size = 2

model = Sequential()
model.add(Dense(256, input_shape=(40,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

num_epochs = 15
num_batch_size = 8

start = datetime.now()

#callbacks=[checkpointer],
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test),  verbose=1)
model.save('models/model4.h5')

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

num_labels 11
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               10496     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 11)                2827      
Total params: 210,699
Trainable params: 210,699
Non-trainable params: 0
_________________________________________________________________
Pre-training accuracy: 14.5161%
Epoch 1/15
31/31 [==============================] - 0s 4ms/step - lo

In [52]:
def classify(filename):
    
    model = load_model('models/model4.h5')

    def extract_feature(file_name):
   
        try:
            audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
            mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
            mfccsscaled = np.mean(mfccs.T,axis=0)
            
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
            return None, None
        return np.array([mfccsscaled])

    #Get class labels for predicted_instrument output
    le = LabelEncoder()
    metadata = pd.read_csv('instruments.csv')
    le.fit(metadata['label'])

    def print_prediction(file_name):
  
        prediction_feature = extract_feature(file_name) 

        predicted_vector = model.predict_classes(prediction_feature)
        #print("predicted_vector", predicted_vector)

        predicted_class = le.inverse_transform(predicted_vector)
        predicted_instrument = predicted_class[0]
        #print("The predicted class is:", predicted_class[0], '\n') 

        predicted_proba_vector = model.predict_proba(prediction_feature) 
        predicted_proba = predicted_proba_vector[0]
        for i in range(len(predicted_proba)):
            category = le.inverse_transform(np.array([i]))
            #print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )
            
        return predicted_instrument
    
    return print_prediction(filename)
    


# The following code was extracted from the unit tests we created.

In [63]:
def predictedInstrumentTest(filename, instrument):

    predicted_instrument = classify(filename)
    print("The predicted instrument is", predicted_instrument, ". The actual instrument is", instrument, ".")
    if (instrument.strip() == predicted_instrument.strip()):
        print("Test passed!!!")
    else:
        print("Test Failed")

In [20]:
!ls wavfiles

 00353774.wav		      88780eac.wav
 0091fc7f.wav		      8916ffcd.wav
 01c2f88b.wav		      8986cced.wav
 0286fc27.wav		      8b6a16de.wav
 0356dec7.wav		      8cd809d3.wav
 03fc1f96.wav		      8ce2e863.wav
 06c48eab.wav		      8d7d7dba.wav
 078f7ac0.wav		      8da75280.wav
 07db7f16.wav		      8ee44fb5.wav
 09a1cfd7.wav		      8fa096bf.wav
 0c67f402.wav		      921eee9d.wav
 0ddb4bdf.wav		      9320c55f.wav
 0e7cd4eb.wav		      95e1d0ae.wav
 0ed06544.wav		      96dad435.wav
 0eeaebcb.wav		      9923ddf8.wav
 103f29e5.wav		      99c96a07.wav
 10da16fb.wav		      9a4bfb69.wav
 1148986e.wav		      9a722e53.wav
 125e6ccf.wav		      9adffd44.wav
 13da16ae.wav		      9c0838c1.wav
 160c92d7.wav		      9cd05a2b.wav
 16c9a838.wav		      9d308738.wav
 17f82c05.wav		      9dd0141c.wav
 18865530.wav		      9e3bc1b6.wav
 19105b8a.wav		      9f522cda.wav
 195f544d.wav		      9f6eac9c.wav
 1980s-Casio-Trumpet-C5.wav   9f84ea5a.wav
 19893aa8.wav		      a053797b.wav
 1a7d3f2d.wav		      a090c45a.wav
 1b24

In [64]:
for file_name in os.listdir('trumpets'):
  predictedInstrumentTest('trumpets/' + file_name, 'trumpet')

The predicted instrument is Saxophone . The actual instrument is trumpet .
#########
trumpet Saxophone
Test Failed
The predicted instrument is  trumpet . The actual instrument is trumpet .
#########
trumpet  trumpet
Test passed!!!
The predicted instrument is Saxophone . The actual instrument is trumpet .
#########
trumpet Saxophone
Test Failed
The predicted instrument is Flute . The actual instrument is trumpet .
#########
trumpet Flute
Test Failed
The predicted instrument is Flute . The actual instrument is trumpet .
#########
trumpet Flute
Test Failed
The predicted instrument is Saxophone . The actual instrument is trumpet .
#########
trumpet Saxophone
Test Failed
The predicted instrument is Saxophone . The actual instrument is trumpet .
#########
trumpet Saxophone
Test Failed
The predicted instrument is  trumpet . The actual instrument is trumpet .
#########
trumpet  trumpet
Test passed!!!
The predicted instrument is Saxophone . The actual instrument is trumpet .
#########
trumpet S